# 04-3 (COMPAT FIXED2)

handoff 出力を仕様に統一し、04-4/04-5 へ確実に引き継げるようにしました。
- 出力: `artifacts/{RUN_ID}/handoff/04-3_llm_tools_setup.pkl`
- 直列化: cloudpickle 優先（fallback あり）
- `cfg/tools/llm` と代表的な `stats` を保存



# 03_ai_agent_analysis_part3_fixed — Config機能追加 & Controller API（改修版）
- セル1: `load_configuration()`（Config読み込み・検証・マージ）
- セル2: Robust handoff loader（Part2受け取り）
- セル3: DB設定管理（cfg/handoff/env優先順位）
- セル4: Handoff展開（必須キー検証）
- セル5: **TLD分布分析**（中核機能を保持したままConfig反映）
- セル6: Handoff保存（pkl + JSON）
- セル7: Controller API `explanation_quality()`（エイリアス `tld_risk_analysis`）


In [1]:
# === Cell 0: 共通ブートストラップ（Strict Mode: _current/run_id.txt 必須版） ===
import os, sys, shutil
from pathlib import Path

def bootstrap_run_id() -> str:
    ROOT = Path(".").resolve()
    
    # ユーザー指定のパス: artifacts/_current/run_id.txt を参照
    # （_current は通常シンボリックリンクまたはディレクトリです）
    id_file = ROOT / "artifacts" / "_current" / "run_id.txt"
    
    # 1. ファイル存在チェック
    if not id_file.exists():
        error_msg = (
            f"❌ クリティカルエラー: 必須ファイルが見つかりません: {id_file}\n"
            "前のステップが実行されていないか、_current ディレクトリ/リンクが作成されていません。\n"
            "処理を停止します。"
        )
        print(f"[NX] {error_msg}")
        # ここで確実に止める
        raise FileNotFoundError(error_msg)

    # 2. ID読み込み
    run_id = id_file.read_text().strip()

    # 3. 空チェック
    if not run_id:
        raise ValueError(f"❌ エラー: {id_file} の中身が空です。")

    print(f"[NX] Current Run ID: {run_id} (loaded from {id_file})")
    
    # 環境変数をセット（後続の互換性維持のため）
    os.environ["RUN_ID"] = run_id
    return run_id

# === 実行 ===
RUN_ID = bootstrap_run_id()

# --- 以下、後続セルのための互換性shim（paths.pyの配置） ---
ROOT = Path(".").resolve()
(Path("_compat")).mkdir(exist_ok=True)

if Path("paths.py").exists():
    target_shim = Path("_compat/paths.py")
    # 内容が異なる場合のみコピー
    if not target_shim.exists() or Path("paths.py").read_text() != target_shim.read_text():
        shutil.copyfile("paths.py", "_compat/paths.py")

Path("_compat/__init__.py").write_text("# shim pkg\n")

from _compat.paths import ensure_roots, compat_base_dirs
ensure_roots()

print(f"[NX] RUN_ID = {RUN_ID} | handoff = {compat_base_dirs['handoff']}")



[NX] Current Run ID: 2026-01-03_222059 (loaded from /data/hdd/asomura/nextstep/artifacts/_current/run_id.txt)
[NX] RUN_ID = 2026-01-03_222059 | handoff = artifacts/2026-01-03_222059/handoff


In [2]:
# === セル1: 環境初期化・パス設定 + load_configuration ===
import os, json, typing, warnings
from pathlib import Path
from datetime import datetime

# --- [FIX START] 修正箇所: pathsモジュールの明示的インポートとBASE_DIRSの定義 ---
try:
    # Cell 0 で生成された _compat パッケージから読み込む
    import _compat.paths as paths
except ImportError:
    # フォールバック
    import paths

# paths.py の compat_base_dirs を BASE_DIRS として参照できるようにする
BASE_DIRS = paths.compat_base_dirs
# --- [FIX END] -------------------------------------------------------------

try:
    import yaml
except Exception:
    yaml = None

RUN_ID = getattr(paths, 'RUN_ID', None)
ARTIFACTS = Path(BASE_DIRS['handoff']).parent

RAW, PROCESSED, MODELS, RESULTS, HANDOFF, LOGS, TRACES = [ARTIFACTS / d for d in ["raw","processed","models","results","handoff","logs","traces"]]
RAW_DIR, PROCESSED_DIR, MODELS_DIR, RESULTS_DIR, HANDOFF_DIR, LOGS_DIR, TRACES_DIR = map(str, [RAW, PROCESSED, MODELS, RESULTS, HANDOFF, LOGS, TRACES])
print(f"✅ IO guard ready -> artifacts/{RUN_ID}")

def _deep_update(base: dict, override: dict) -> dict:
    base = dict(base or {})
    for k, v in (override or {}).items():
        if isinstance(v, dict) and isinstance(base.get(k), dict):
            base[k] = _deep_update(base[k], v)
        else:
            base[k] = v
    return base

from typing import Optional, Dict, Any
def load_configuration(config_path: Optional[str] = None,
                       cfg_override: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    defaults = {
        "system": {"cert_only_mode": True, "seed": 42},
        "db": {"dbname":"rapids_data","user":"postgres","password":"asomura","host":"localhost","port":"5432","timeout_s":30,"read_only":True},
        "tld_analysis": {"enabled":True,"use_all_data":True,"balance_data":True,"sample_size":None,"min_sample_size":10,
                         "max_dangerous_tlds":30,"max_legitimate_tlds":30,"percentile_thresholds":[50,75,90],"fn_weight":1.5},
        "engine": {"max_concurrent":20,"max_retries":3,"batch_size":10000},
        "paths": {"base_dir":"artifacts"},
        "llm": {"enabled":False}
    }
    cfg = dict(defaults)
    
    # config.json のパス補正（_compatにある場合）
    if not config_path and Path("_compat/config.json").exists():
        config_path = "_compat/config.json"
        
    if config_path and Path(config_path).exists():
        try:
            text = Path(config_path).read_text(encoding="utf-8")
            if Path(config_path).suffix.lower() in (".yml",".yaml") and yaml:
                cfg = _deep_update(cfg, yaml.safe_load(text) or {})
            else:
                cfg = _deep_update(cfg, json.loads(text))
        except Exception as e:
            warnings.warn(f"設定ファイル読込失敗: {e}")
    env_map = {"USE_ALL_DATA":("tld_analysis","use_all_data"),
               "BALANCE_DATA":("tld_analysis","balance_data"),
               "PGDATABASE":("db","dbname"),
               "PGUSER":("db","user"),
               "PGPASSWORD":("db","password")}
    for env, path in env_map.items():
        if env in os.environ and os.environ[env]!="":
            val = os.environ[env]
            if isinstance(val,str) and val.lower() in ("true","false"):
                val = (val.lower()=="true")
            node = cfg
            for k in path[:-1]:
                node = node.setdefault(k,{})
            node[path[-1]] = val
    if isinstance(cfg_override, dict):
        cfg = _deep_update(cfg, cfg_override)
    if not cfg["system"].get("cert_only_mode", False):
        raise ValueError("system.cert_only_mode は True 必須です")
    if not cfg.get("tld_analysis",{}).get("enabled", False):
        raise ValueError("tld_analysis.enabled は True 必須です")
    for k in ("dbname","user","host","port"):
        if not str(cfg["db"].get(k,"")).strip():
            raise ValueError(f"DB必須項目不足: db.{k}")
    globals()["cfg"] = cfg
    print("🔧 Config loaded (cert_only_mode=True, tld_analysis.enabled=True)")
    return cfg

# 修正前:
# cfg = load_configuration(os.getenv("CONFIG_PATH"), None)

# 修正後: system.cert_only_mode を強制的に True に設定して読み込む
cfg = load_configuration(os.getenv("CONFIG_PATH"), {"system": {"cert_only_mode": True}})

✅ IO guard ready -> artifacts/2026-01-03_222059
🔧 Config loaded (cert_only_mode=True, tld_analysis.enabled=True)


In [3]:
# === Cell 4 (FIXED): Robust handoff loader (04-2 → 04-3) ===
import joblib, glob, warnings
import pandas as pd
from pathlib import Path

# 保存先ディレクトリの確保
Path(HANDOFF_DIR).mkdir(parents=True, exist_ok=True)

# ✅ 正しい候補リスト（04-2 の出力を優先）
cands = [
    str(Path(HANDOFF_DIR) / "04-2_statistical_analysis.pkl"),
    "handoff/04-2_statistical_analysis.pkl",
]

# もし見つからない場合は artifacts/*/handoff から探索
if not any(Path(p).exists() for p in cands):
    cands += sorted(glob.glob("artifacts/*/handoff/04-2_statistical_analysis.pkl"))

src = next((p for p in cands if Path(p).exists()), None)
if not src:
    raise FileNotFoundError("04-2 handoff が見つかりません。")

print(f"📂 Loading handoff from: {src}")

# === 修正ポイント: バージョン不整合対策 ===
handoff = {}
try:
    # 通常のロード試行
    handoff = joblib.load(src)
    print(f"✅ handoff loaded successfully.")
except (TypeError, ValueError, ImportError) as e:
    # Pythonバージョン違い(3.10/3.11 -> 3.12)などでロード失敗した場合のフォールバック
    warnings.warn(f"⚠️ Pickle load failed due to version mismatch: {e}")
    print("🔄 Attempting fallback reconstruction from CSV/Defaults...")

    # 1. false_negatives_df をCSVから復元（あれば）
    # 通常 artifacts/{RUN_ID}/results/false_negatives.csv にある
    fn_csv = Path(src).parent.parent / "results" / "false_negatives.csv"
    if fn_csv.exists():
        try:
            handoff['false_negatives_df'] = pd.read_csv(fn_csv)
            print(f"  - Recovered 'false_negatives_df' from {fn_csv.name}")
        except Exception as csv_e:
            print(f"  - Failed to read CSV: {csv_e}")
            handoff['false_negatives_df'] = pd.DataFrame(columns=['domain','ml_probability'])
    else:
        print("  - CSV not found. Initializing empty 'false_negatives_df'.")
        handoff['false_negatives_df'] = pd.DataFrame(columns=['domain','ml_probability'])

    # 2. その他の必須キーをデフォルト値で初期化
    # configからキーワード取得を試みる
    default_keywords = cfg.get("brand_keywords", ["paypal", "mercari", "amazon", "apple", "google"])
    if isinstance(default_keywords, dict): # 辞書ならキーリスト化
        default_keywords = list(default_keywords.keys())
    
    handoff['brand_keywords'] = default_keywords
    handoff['cert_full_info_map'] = {}  # 証明書情報は空で初期化
    handoff['fn_features_df'] = pd.DataFrame()
    
    print("✅ Fallback handoff created with default values.")

# 内容確認
print(f"✅ keys: {list(handoff.keys())[:10]} ...")

📂 Loading handoff from: artifacts/2026-01-03_222059/handoff/04-2_statistical_analysis.pkl
✅ handoff loaded successfully.
✅ keys: ['cfg', 'RUN_ID', 'SESSION_ID', 'output_dirs', 'brand_keywords', 'cert_full_info_map', 'false_negatives_df', 'fn_features_df', 'HIGH_RISK_WORDS', 'suspicious_words_stats'] ...


In [4]:

# === セル3: DB設定管理 ===
import os
def _db_from_env():
    return {'dbname':os.getenv('PGDATABASE','rapids_data'),
            'user':os.getenv('PGUSER','postgres'),
            'password':os.getenv('PGPASSWORD','asomura'),
            'host':os.getenv('PGHOST','localhost'),
            'port':os.getenv('PGPORT','5432')}
def build_db_config(cfg: dict, handoff: dict) -> dict:
    if isinstance(cfg, dict) and 'db' in cfg:
        base = {k:str(cfg['db'].get(k)) for k in ['dbname','user','password','host','port']}
    elif isinstance(handoff, dict) and 'DB_CONFIG' in handoff:
        base = dict(handoff['DB_CONFIG'])
    else:
        base = _db_from_env()
    base['connect_timeout'] = int((cfg.get('db',{}) or {}).get('timeout_s',30)) if isinstance(cfg,dict) else 30
    base['_read_only'] = bool((cfg.get('db',{}) or {}).get('read_only',True)) if isinstance(cfg,dict) else True
    return base
DB_CONFIG = build_db_config(cfg, handoff)
print("🔧 DB_CONFIG ready:", {k: DB_CONFIG[k] for k in ['dbname','host','port','user']}, f"(timeout={DB_CONFIG['connect_timeout']}s)")


🔧 DB_CONFIG ready: {'dbname': 'rapids_data', 'host': 'localhost', 'port': '5432', 'user': 'postgres'} (timeout=30s)


In [5]:

# === セル4: Handoff展開 ===
required = ['false_negatives_df','brand_keywords','cert_full_info_map','fn_features_df']
miss = [k for k in required if k not in handoff]
if miss: raise KeyError(f"handoff keys missing: {miss}")
false_negatives_df = handoff['false_negatives_df']
brand_keywords = handoff['brand_keywords']
cert_full_info_map = handoff['cert_full_info_map']
fn_features_df = handoff['fn_features_df']
if 'DB_CONFIG' in handoff: DB_CONFIG = {**DB_CONFIG, **handoff['DB_CONFIG']}
print(f"✅ unpack: fn_rows={getattr(false_negatives_df,'shape',[None,None])[0]}, brand_keywords={len(brand_keywords) if isinstance(brand_keywords,(list,dict,set,tuple)) else 'n/a'}")


✅ unpack: fn_rows=7710, brand_keywords=100


In [6]:
# === Cell 6.5: Promote handoff keys to UPPER aliases & guard types ===
# 前提: Cell6 で handoff を小文字の変数に展開済み
# 目的: 後続 writer セルが参照する大文字名(BRAND_KEYWORDS, CERT_FULL_INFO_MAP, TOOLS_DICT)を必ず定義する

# 1) brand_keywords -> BRAND_KEYWORDS
if 'brand_keywords' in globals():
    if isinstance(brand_keywords, (set, tuple)):
        BRAND_KEYWORDS = list(brand_keywords)
    elif isinstance(brand_keywords, list):
        BRAND_KEYWORDS = brand_keywords
    else:
        try:
            BRAND_KEYWORDS = list(brand_keywords)
        except Exception:
            BRAND_KEYWORDS = []
            print("[04-3][WARN] brand_keywords の型が想定外だったため空で代替。")
else:
    BRAND_KEYWORDS = handoff.get('brand_keywords', [])

# 2) cert_full_info_map -> CERT_FULL_INFO_MAP
if 'cert_full_info_map' in globals():
    if not isinstance(cert_full_info_map, dict):
        try:
            CERT_FULL_INFO_MAP = dict(cert_full_info_map)
        except Exception:
            CERT_FULL_INFO_MAP = {}
            print("[04-3][WARN] cert_full_info_map の型が想定外だったため空で代替。")
    else:
        CERT_FULL_INFO_MAP = cert_full_info_map
else:
    CERT_FULL_INFO_MAP = handoff.get('cert_full_info_map', {})

# 3) TOOLS_DICT の初期化（04-2 handoff には通常含まれない）
if 'TOOLS_DICT' not in globals():
    TOOLS_DICT = handoff.get('tools', {})
    if not isinstance(TOOLS_DICT, dict):
        TOOLS_DICT = {}

print(f"[04-3][CHECK] aliases -> BRAND_KEYWORDS={len(BRAND_KEYWORDS)}, "
      f"CERT_FULL_INFO_MAP={len(CERT_FULL_INFO_MAP)}, TOOLS_DICT={len(TOOLS_DICT)}")


[04-3][CHECK] aliases -> BRAND_KEYWORDS=100, CERT_FULL_INFO_MAP=7710, TOOLS_DICT=0


In [7]:

# === セル5: TLD分布分析（中核保持 + Config） ===
import psycopg2
from psycopg2.extras import RealDictCursor
from collections import Counter
from urllib.parse import urlparse
import numpy as np, random

ta = cfg.get('tld_analysis', {})
use_all_data   = bool(ta.get('use_all_data', True))
balance_data   = bool(ta.get('balance_data', True))
sample_size    = ta.get('sample_size', None)
min_sample     = int(ta.get('min_sample_size', 10))
max_dangerous  = int(ta.get('max_dangerous_tlds', 30))
max_legitimate = int(ta.get('max_legitimate_tlds', 30))
pct_thresholds = list(ta.get('percentile_thresholds', [50,75,90]))
fn_weight      = float(ta.get('fn_weight', 1.5))
random.seed(cfg.get('system',{}).get('seed',42))

def extract_tld(domain):
    if not domain: return None
    if '://' in domain:
        try: domain = urlparse(domain).netloc
        except Exception: pass
    domain = domain.split(':')[0]
    parts = domain.split('.')
    if len(parts) >= 2:
        if len(parts) >= 3 and parts[-2] in ['co','ac','or','ne','go']:
            return f'.{parts[-2]}.{parts[-1]}'
        return f'.{parts[-1]}'
    return None

conn = psycopg2.connect(dbname=DB_CONFIG['dbname'], user=DB_CONFIG['user'], password=DB_CONFIG['password'],
                        host=DB_CONFIG['host'], port=DB_CONFIG['port'],
                        connect_timeout=int(DB_CONFIG.get('connect_timeout',30)))
cur = conn.cursor(cursor_factory=RealDictCursor)
print("✅ DB connected")

phishing_queries = {
    'phishtank': "SELECT cert_domain as domain FROM phishtank_entries WHERE cert_status='SUCCESS' AND cert_data IS NOT NULL AND cert_domain IS NOT NULL",
    'jpcert':    "SELECT domain FROM jpcert_phishing_urls WHERE status='SUCCESS' AND cert_data IS NOT NULL AND domain IS NOT NULL",
    'certificates': "SELECT domain FROM certificates WHERE status='SUCCESS' AND cert_data IS NOT NULL AND domain IS NOT NULL"
}
ph_dom = []; ph_tlds = Counter()
for name, q in phishing_queries.items():
    if not use_all_data and sample_size: q += f" LIMIT {int(sample_size)}"
    cur.execute(q); rows = cur.fetchall()
    st = Counter()
    for r in rows:
        d = r['domain']
        if d:
            ph_dom.append(d)
            t = extract_tld(d)
            if t:
                st[t]+=1; ph_tlds[t]+=1
    print(f"  {name}: {len(rows):,}件 (Top5: {', '.join([f'{t}({c})' for t,c in st.most_common(5)])})")

q = "SELECT domain FROM trusted_certificates WHERE status='SUCCESS' AND cert_data IS NOT NULL AND domain IS NOT NULL"
if not use_all_data and sample_size: q += f" LIMIT {int(sample_size)}"
cur.execute(q); rows = cur.fetchall()
tr_dom = []; tr_tlds = Counter()
for r in rows:
    d = r['domain']
    if d:
        tr_dom.append(d)
        t = extract_tld(d)
        if t: tr_tlds[t]+=1

if balance_data:
    minc = min(len(set(ph_dom)), len(set(tr_dom)))
    ph_u = list(set(ph_dom)); tr_u = list(set(tr_dom))
    random.seed(cfg.get('system',{}).get('seed',42))
    ph_bal = random.sample(ph_u, minc) if len(ph_u)>minc else ph_u
    tr_bal = random.sample(tr_u, minc) if len(tr_u)>minc else tr_u
else:
    ph_bal, tr_bal = ph_dom, tr_dom

ph_bal_t = Counter(extract_tld(d) for d in ph_bal if extract_tld(d))
tr_bal_t = Counter(extract_tld(d) for d in tr_bal if extract_tld(d))

from collections import Counter as _Ctr
fn_tlds = _Ctr()
if hasattr(false_negatives_df,'columns') and 'domain' in false_negatives_df.columns:
    for d in false_negatives_df['domain']:
        t = extract_tld(d)
        if t: fn_tlds[t]+=1

dangerous_ratio = {}
for t in ph_bal_t:
    ph = ph_bal_t[t]; tr = tr_bal_t.get(t,0)
    if ph >= min_sample:
        ratio = float('inf') if tr==0 else ph/(tr+1)
        dangerous_ratio[t] = {'ratio':ratio,'phishing_count':ph,'trusted_count':tr,
                              'phishing_pct': ph/max(len(ph_bal),1)*100}

ratios = [s['ratio'] for s in dangerous_ratio.values() if s['ratio']!=float('inf')]
if ratios:
    pcts = np.percentile(ratios, pct_thresholds).tolist()
else:
    pcts = [10,10,10]
p50,p75,p90 = (pcts+[10,10,10])[:3]

DANGEROUS_TLDS = []
for t,s in sorted(dangerous_ratio.items(), key=lambda x:x[1]['ratio'], reverse=True):
    if ((s['ratio']==float('inf') and s['phishing_count']>=10) or
        (s['ratio']>=p90 and s['phishing_pct']>=0.1) or
        (s['ratio']>=10 and s['phishing_pct']>=1.0)):
        DANGEROUS_TLDS.append(t)
        if len(DANGEROUS_TLDS)>=max_dangerous: break

LEGITIMATE_TLDS = []
for t,c in tr_bal_t.most_common(max_legitimate*2):
    tr_pct = c/max(len(tr_bal),1)*100
    if tr_pct>=1.0 and c>=1000:
        ph = ph_bal_t.get(t,0)
        ratio = ph/c if c>0 else 0.0
        if ratio<0.5:
            LEGITIMATE_TLDS.append(t)
    if len(LEGITIMATE_TLDS)>=max_legitimate: break

if len(fn_tlds):
    for t,c in fn_tlds.most_common():
        fn_pct = c/max(len(false_negatives_df),1)*100
        if fn_pct>=1.0 and t not in DANGEROUS_TLDS and t not in LEGITIMATE_TLDS:
            if t in dangerous_ratio and dangerous_ratio[t]['ratio']>=5:
                DANGEROUS_TLDS.append(t)

NEUTRAL_TLDS = []
for t in ['.com','.org','.net','.info','.biz']:
    if t not in DANGEROUS_TLDS and t not in LEGITIMATE_TLDS:
        if t in ph_bal_t and t in tr_bal_t:
            ph_pct = ph_bal_t[t]/max(len(ph_bal),1)*100
            tr_pct = tr_bal_t[t]/max(len(tr_bal),1)*100
            if ph_pct>=0.5 or tr_pct>=0.5:
                NEUTRAL_TLDS.append(t)

globals().update({'DANGEROUS_TLDS':DANGEROUS_TLDS,'LEGITIMATE_TLDS':LEGITIMATE_TLDS,'NEUTRAL_TLDS':NEUTRAL_TLDS,
                  'phishing_tld_stats':ph_bal_t,'trusted_tld_stats':tr_bal_t})
cur.close(); conn.close()
print(f"✅ TLD分析完了: dangerous={len(DANGEROUS_TLDS)}, legitimate={len(LEGITIMATE_TLDS)}, neutral={len(NEUTRAL_TLDS)}")


✅ DB connected
  phishtank: 54,362件 (Top5: .com(23196), .dev(5865), .ly(3191), .me(2732), .de(2705))
  jpcert: 116,647件 (Top5: .com(35795), .cn(34513), .top(6972), .org(5610), .shop(4572))
  certificates: 196,392件 (Top5: .org(102083), .com(30237), .cn(27498), .top(7450), .xyz(6099))
✅ TLD分析完了: dangerous=24, legitimate=8, neutral=1


In [8]:
# === セル6: Handoff保存（JSONのみ；主PKL保存は FINAL v3 セルで実施） ===
import json, time
from pathlib import Path

# Note: legacy write to 03_ai_agent_analysis_part3.pkl is intentionally disabled to avoid cross-module overwrite.
# The single-file handoff PKL is saved in the FINAL v3 cell as "04-3_llm_tools_setup_with_tools.pkl".

Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)
with open(Path(RESULTS_DIR)/"tld_statistics.json", "w", encoding="utf-8") as f:
    json.dump({'ts': time.time(),
               'dangerous_tlds': list(DANGEROUS_TLDS),
               'legitimate_tlds': list(LEGITIMATE_TLDS),
               'neutral_tlds': list(NEUTRAL_TLDS)}, f, ensure_ascii=False, indent=2)

print("[04-3] Skipped legacy part3.pkl write (by design).")
print(f"[04-3] Saved JSON: {Path(RESULTS_DIR)/'tld_statistics.json'}")


[04-3] Skipped legacy part3.pkl write (by design).
[04-3] Saved JSON: artifacts/2026-01-03_222059/results/tld_statistics.json


In [9]:

# === セル7: Controller API関数 ===
from typing import Tuple, Dict, Any
import traceback
from pathlib import Path

def explanation_quality(session_id: str, cfg: Dict[str, Any]) -> Tuple[str, Dict[str, str]]:
    try:
        if not isinstance(cfg, dict):
            return "INVALID_INPUT", {"error":"cfgは辞書型で指定してください"}
        _ = load_configuration(cfg_override=cfg)
        if not _.get('tld_analysis',{}).get('enabled', False):
            return "INVALID_INPUT", {"error":"tld_analysis.enabled が無効"}
        if not Path(HANDOFF_DIR, "03_ai_agent_analysis_part2.pkl").exists():
            return "NOT_FOUND", {"error":"Part2 handoff が見つかりません"}
        paths = {
            "dangerous_tlds": str(Path(RESULTS_DIR)/"tld_statistics.json"),
            "legitimate_tlds": str(Path(RESULTS_DIR)/"tld_statistics.json"),
            "neutral_tlds": str(Path(RESULTS_DIR)/"tld_statistics.json"),
            "tld_statistics": str(Path(RESULTS_DIR)/"tld_statistics.json"),
            "handoff": str(Path(HANDOFF_DIR)/"03_ai_agent_analysis_part3.pkl"),
            "logs": LOGS_DIR
        }
        if len(DANGEROUS_TLDS)==0 and len(LEGITIMATE_TLDS)==0:
            return "NOT_FOUND", {"error":"TLDリストが空です", **paths}
        return "OK", paths
    except Exception as e:
        return "ERROR", {"error": str(e), "trace": traceback.format_exc()}

tld_risk_analysis = explanation_quality
print("✅ Controller API ready: explanation_quality() / tld_risk_analysis()")


✅ Controller API ready: explanation_quality() / tld_risk_analysis()


In [10]:
# === P1-2: brand_impersonation_check (compat wrapper; Structured Output) ===
from typing import Dict, Any, Iterable, List, Tuple, Optional
import re

_P1_2_RISK_WORDS = {'secure','login','verify','update','account','wallet','support','confirm','bank','invoice','payment'}

def _p1_2_risk_level_from_confidence(conf: float) -> str:
    if conf >= 0.85: return "critical"
    if conf >= 0.70: return "high"
    if conf >= 0.50: return "medium-high"
    if conf >= 0.30: return "medium"
    return "low"

def _p1_2_iter_keywords(bk: Any):
    if bk is None: return []
    if isinstance(bk, dict):
        for k, v in bk.items():
            if isinstance(k, str) and k: yield k.lower()
            if isinstance(v, (list, tuple, set)):
                for x in v:
                    if isinstance(x, str) and x: yield x.lower()
            elif isinstance(v, str) and v: yield v.lower()
        return
    if isinstance(bk, (list, tuple, set)):
        for x in bk:
            if isinstance(x, str) and x: yield x.lower()
        return
    try:
        for x in list(bk):
            if isinstance(x, str) and x: yield x.lower()
    except Exception:
        return []

def _p1_2_tokenize_domain(d: str) -> List[str]:
    try: d = d.encode('idna').decode('ascii')
    except Exception: pass
    return [t for t in re.split(r'[^a-z0-9]+', (d or '').lower()) if t]

def _p1_2_is_one_edit(a: str, b: str) -> bool:
    la, lb = len(a), len(b)
    if abs(la - lb) > 1: return False
    if la > lb: a, b, la, lb = b, a, lb, la
    i = j = edits = 0
    while i < la and j < lb:
        if a[i] == b[j]: i += 1; j += 1
        else:
            edits += 1
            if edits > 1: return False
            if la == lb: i += 1; j += 1
            else: j += 1
    edits += (lb - j) + (la - i)
    return edits <= 1

def brand_impersonation_check(domain: str, ml_probability: float = 0.0, cfg: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    d = (domain or "").strip().lower()
    try: p = float(ml_probability or 0.0)
    except Exception: p = 0.0

    g = globals()
    keywords_src = g.get("BRAND_KEYWORDS", g.get("brand_keywords", []))
    brands = sorted(set(_p1_2_iter_keywords(keywords_src)))
    tokens = _p1_2_tokenize_domain(d)

    substr_hits = [b for b in brands if b and b in d]
    token_hits  = [b for b in brands if b in tokens]

    typo_hits: List[Tuple[str, str]] = []
    cand = [t for t in tokens if len(t) >= 3]
    for b in brands:
        if not b or len(b) < 4: continue
        if b in substr_hits or b in token_hits: continue
        for t in cand:
            if _p1_2_is_one_edit(b, t):
                typo_hits.append((b, t))
                break

    risk_kw = [kw for kw in _P1_2_RISK_WORDS if kw in d]

    score = 0.0
    if substr_hits or token_hits: score += 0.60
    if typo_hits: score += 0.25
    if risk_kw: score += 0.10 + 0.02 * min(len(risk_kw), 5)
    score += 0.30 * p
    score = max(0.0, min(score, 0.99))

    return {
        "is_impersonation": bool(substr_hits or token_hits or typo_hits),
        "confidence": score,
        "risk_level": _p1_2_risk_level_from_confidence(score),
        "detected_brands": sorted(set(substr_hits + token_hits)),
        "risk_factors": {
            "typosquatting_matches": typo_hits,
            "risk_keywords": risk_kw,
            "ml_probability": p,
            "tokens": tokens[:8],
        },
        "reasons": [
            "brand keyword present" if (substr_hits or token_hits) else "no direct brand keyword found",
            "typosquatting candidate detected" if typo_hits else "no typo candidate",
            f"ml_probability contribution={round(0.30*p,3)}"
        ],
        "success": True,
    }

# Register into TOOLS_DICT for inventory
try:
    if "TOOLS_DICT" not in globals() or not isinstance(TOOLS_DICT, dict):
        TOOLS_DICT = {}
    TOOLS_DICT["brand_impersonation_check"] = {
        "type": "tool", "version": "p1-2",
        "inputs": ["domain", "ml_probability"],
        "returns": ["is_impersonation","confidence","risk_level","detected_brands","risk_factors","reasons","success"],
        "notes": "Local wrapper; uses BRAND_KEYWORDS when available; no external I/O."
    }
    print("✅ tool registered: brand_impersonation_check")
except Exception as _e:
    print("⚠️ tool registration failed:", _e)


✅ tool registered: brand_impersonation_check


In [11]:

# Cell 98: Sanity (keys expected to be produced by 04-3)
_expect = ["brand_keywords","cert_full_info_map","HIGH_RISK_WORDS",
           "brand_domains_map","cert_anomaly_rules","stat_feature_defs"]
present = [k for k in _expect if k in globals()]
missing = [k for k in _expect if k not in globals()]
print("[04-3][SANITY] present:", present)
if missing:
    print("[04-3][SANITY][WARN] missing (ensure these are produced upstream in this notebook):", missing)


[04-3][SANITY] present: ['brand_keywords', 'cert_full_info_map']
[04-3][SANITY][WARN] missing (ensure these are produced upstream in this notebook): ['HIGH_RISK_WORDS', 'brand_domains_map', 'cert_anomaly_rules', 'stat_feature_defs']


In [12]:

# Cell 99: Save handoff (robust cloudpickle-first)
import os, json
from pathlib import Path
try:
    import cloudpickle as _cp
except Exception:
    _cp = None
import pickle as _sp

def _is_picklable(x):
    try:
        if _cp is not None:
            _cp.dumps(x)
        else:
            _sp.dumps(x)
        return True
    except Exception:
        return False

payload = {}
payload["cfg"]   = globals().get("cfg")
payload["tools"] = globals().get("tools")
payload["llm"]   = globals().get("llm")

stats = {}
maybe_names = [
    "brand_keywords","cert_full_info_map","HIGH_RISK_WORDS",
    "brand_domains_map","cert_anomaly_rules","stat_feature_defs",
    "tool_specs","llm_params","tokenizer_info"
]
for name in maybe_names:
    if name in globals():
        stats[name] = globals()[name]

if "stats" in globals() and isinstance(globals()["stats"], dict):
    for k, v in globals()["stats"].items():
        if k not in stats:
            stats[k] = v

payload["stats"] = stats

out_dir = Path(f"artifacts/{RUN_ID}/handoff")
out_dir.mkdir(parents=True, exist_ok=True)
out_pkl = out_dir / "04-3_llm_tools_setup.pkl"

def _try_dump(obj, path, use_cloud=True):
    if use_cloud and _cp is not None:
        with open(path, "wb") as f:
            _cp.dump(obj, f)
        return "cloudpickle"
    else:
        with open(path, "wb") as f:
            _sp.dump(obj, f)
        return "pickle"

ok = False
try:
    m = _try_dump(payload, out_pkl, use_cloud=True)
    ok = True
    print(f"[04-3][SAVE] Saved ({m}) ->", out_pkl)
except Exception as e:
    print("[04-3][WARN] cloudpickle save failed for whole payload:", e)
    # prune unpicklables
    for k in list(payload.keys()):
        try:
            if _cp is not None:
                _cp.dumps(payload[k])
            else:
                _sp.dumps(payload[k])
        except Exception:
            print("  - pruning top-level:", k)
            payload[k] = None
    for k in list(stats.keys()):
        try:
            if _cp is not None:
                _cp.dumps(stats[k])
            else:
                _sp.dumps(stats[k])
        except Exception:
            print("  - pruning stats:", k)
            stats[k] = None
    try:
        m = _try_dump(payload, out_pkl, use_cloud=(_cp is not None))
        ok = True
        print(f"[04-3][SAVE] Saved (fallback:{m}) ->", out_pkl)
    except Exception as e2:
        print("[04-3][FATAL] save failed:", e2)

# manifest
man = {"run_id": RUN_ID, "path": str(out_pkl), "top_level": sorted(list(payload.keys())), "stats_keys": sorted(list(stats.keys()))}
try:
    with open(out_dir / "04-3_manifest.json", "w", encoding="utf-8") as mf:
        json.dump(man, mf, ensure_ascii=False, indent=2)
    print("[04-3][SAVE] manifest ->", out_dir / "04-3_manifest.json")
except Exception as _:
    pass

if ok:
    sz = out_pkl.stat().st_size if out_pkl.exists() else 0
    print(f"[04-3][SAVE] output size = {sz} bytes")
    assert sz > 0, "handoff file is empty (0 bytes)"


[04-3][SAVE] Saved (cloudpickle) -> artifacts/2026-01-03_222059/handoff/04-3_llm_tools_setup.pkl
[04-3][SAVE] manifest -> artifacts/2026-01-03_222059/handoff/04-3_manifest.json
[04-3][SAVE] output size = 1023748 bytes


In [13]:
# === P1-3: certificate_analysis (compat wrapper; Structured Output) ===
# Behavior:
#  - Uses CERT_FULL_INFO_MAP when available; otherwise degrades gracefully
#  - Heuristics per paper (validity days, issuer/free CA, wildcard, SAN count, self-signed, org presence)
#  - Returns fields consumed by certificate_analysis_node in 04-4
from typing import Dict, Any, Optional, List, Tuple
from datetime import datetime

_FREE_CA_HINTS = ("let's encrypt","zerossl","buypass","trustasia","gandi","cloudflare","google internet authority","isrg","actalis")
def _p1_3_risk_level(score: float) -> str:
    if score >= 80: return "critical"
    if score >= 60: return "high"
    if score >= 45: return "medium-high"
    if score >= 25: return "medium"
    return "low"

def _p1_3_norm_dt(s: Any) -> Optional[datetime]:
    if not s: return None
    if isinstance(s, datetime): return s
    if isinstance(s, (int, float)): 
        try:
            # seconds epoch
            return datetime.utcfromtimestamp(float(s))
        except Exception:
            return None
    # string
    s = str(s).strip().replace("Z", "")
    for fmt in ("%Y-%m-%dT%H:%M:%S", "%Y-%m-%d %H:%M:%S", "%Y-%m-%d"):
        try:
            return datetime.strptime(s, fmt)
        except Exception:
            pass
    try:
        # ISO 8601 fallback
        return datetime.fromisoformat(s)
    except Exception:
        return None

def _p1_3_days_between(a: Any, b: Any) -> Optional[int]:
    da, db = _p1_3_norm_dt(a), _p1_3_norm_dt(b)
    if da and db:
        try:
            return abs((db - da).days)
        except Exception:
            return None
    return None

def _p1_3_pick_cert_record(cfmap: Any, domain: str) -> Dict[str, Any]:
    if isinstance(cfmap, dict):
        # direct
        rec = cfmap.get(domain) or cfmap.get(domain.lower())
        if rec: return rec if isinstance(rec, dict) else {}
        # try www/no-www variants
        if domain.startswith("www."):
            rec = cfmap.get(domain[4:])
            if rec: return rec if isinstance(rec, dict) else {}
        else:
            rec = cfmap.get("www."+domain)
            if rec: return rec if isinstance(rec, dict) else {}
        # fuzzy: try eTLD+1 fallback (simple split)
        parts = domain.split(".")
        if len(parts) >= 2:
            etld1 = ".".join(parts[-2:])
            rec = cfmap.get(etld1)
            if rec: return rec if isinstance(rec, dict) else {}
    return {}

def certificate_analysis(domain: str, ml_probability: float = 0.0, cfg: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    d = (domain or "").strip().lower()
    try:
        p = float(ml_probability or 0.0)
    except Exception:
        p = 0.0

    g = globals()
    cfmap = g.get("CERT_FULL_INFO_MAP") or g.get("cert_full_info_map") or {}
    rec = _p1_3_pick_cert_record(cfmap, d)

    # Extract fields (robustly)
    issuer = (str(rec.get("issuer") or rec.get("issuer_cn") or rec.get("ca") or "")).strip()
    org    = (str(rec.get("organization") or rec.get("org") or "")).strip()
    valid_days = rec.get("valid_days")
    if valid_days is None:
        valid_days = _p1_3_days_between(rec.get("valid_from") or rec.get("not_before"),
                                        rec.get("valid_to")   or rec.get("not_after"))
    try:
        valid_days = int(valid_days) if valid_days is not None else None
    except Exception:
        valid_days = None

    san_count = rec.get("san_count")
    if san_count is None:
        sans = rec.get("subject_alt_names") or rec.get("dns_names") or rec.get("san") or []
        try: san_count = len(sans)
        except Exception: san_count = None

    self_signed = bool(rec.get("self_signed") or rec.get("is_self_signed", False))
    wildcard    = bool(rec.get("wildcard") or rec.get("is_wildcard", False))

    issuer_lc = issuer.lower()
    free_ca = any(h in issuer_lc for h in _FREE_CA_HINTS) if issuer else False
    very_new = (valid_days is not None and valid_days <= 120)

    # Score (bounded)
    score = 0.0
    if self_signed: score += 35
    if not org:     score += 15
    if free_ca:     score += 10
    if very_new:    score += 10
    if wildcard:    score += 5
    if isinstance(san_count, int) and san_count <= 1: score += 5
    score += 8.0 * float(p or 0.0)  # ML probability contributes modestly
    score = max(0.0, min(score, 99.0))

    level = _p1_3_risk_level(score)
    conf  = round(min(1.0, max(0.0, (score / 100.0) + 0.10 * float(p or 0.0))), 3)

    reasons: List[str] = []
    if self_signed: reasons.append("self-signed certificate")
    if not org:     reasons.append("organization field missing (DV)")
    if free_ca:     reasons.append("issued by widely-used free CA")
    if very_new:    reasons.append(f"short validity ({valid_days} days)" if valid_days is not None else "short validity period")
    if wildcard:    reasons.append("wildcard certificate detected")
    if isinstance(san_count, int) and san_count <= 1: reasons.append("SAN count is small")
    if not reasons:
        reasons.append("no strong anomalies found in certificate metadata")

    result = {
        "confidence": conf,
        "risk_level": level,
        "risk_factors": {
            "issuer": issuer or None,
            "organization_present": bool(org),
            "valid_days": valid_days,
            "san_count": san_count,
            "self_signed": self_signed,
            "wildcard": wildcard,
            "free_ca": free_ca,
            "very_new_cert": very_new,
            "ml_probability": p,
        },
        "reasons": reasons,
        "success": True,
    }
    return result

# Register into TOOLS_DICT for inventory/handoff
try:
    if "TOOLS_DICT" not in globals() or not isinstance(TOOLS_DICT, dict):
        TOOLS_DICT = {}
    TOOLS_DICT["certificate_analysis"] = {
        "type": "tool", "version": "p1-3",
        "inputs": ["domain", "ml_probability"],
        "returns": ["confidence","risk_level","risk_factors","reasons","success"],
        "notes": "Local wrapper; uses CERT_FULL_INFO_MAP when available; no external I/O."
    }
    print("✅ tool registered: certificate_analysis")
except Exception as _e:
    print("⚠️ tool registration failed:", _e)


✅ tool registered: certificate_analysis


In [14]:
# === P1-3 TEST (local, no external I/O) ===
# Prepare a minimal cert map only if not provided upstream
if not globals().get("CERT_FULL_INFO_MAP"):
    CERT_FULL_INFO_MAP = {
        "paypal-secure-login.info": {
            "issuer": "Let's Encrypt",
            "organization": "",
            "valid_days": 90,
            "san_count": 1,
            "self_signed": False,
            "wildcard": False
        }
    }
out = certificate_analysis("paypal-secure-login.info", 0.34)
print("P1-3 test result:", out)
assert isinstance(out, dict) and "confidence" in out and "risk_level" in out and "risk_factors" in out and "success" in out
assert isinstance(out["risk_factors"], dict) and "free_ca" in out["risk_factors"]
print("P1-3 smoketest: OK")


P1-3 test result: {'confidence': 0.261, 'risk_level': 'low', 'risk_factors': {'issuer': None, 'organization_present': False, 'valid_days': None, 'san_count': 0, 'self_signed': False, 'wildcard': False, 'free_ca': False, 'very_new_cert': False, 'ml_probability': 0.34}, 'reasons': ['organization field missing (DV)', 'SAN count is small'], 'success': True}
P1-3 smoketest: OK


In [15]:

# === Cell [FINAL v3]: Handoff save (single-file, robust, _compat.paths + runreg) ===
from __future__ import annotations
import os, io, json, time, traceback
from pathlib import Path

# Prefer cloudpickle; fallback to pickle
try:
    import cloudpickle as _cp
except Exception:
    _cp = None
import pickle as _pickle

def _import_paths():
    """Prefer `_compat.paths`, then `paths`, then file-based fallbacks."""
    try:
        import _compat.paths as _paths
        return _paths
    except Exception:
        pass
    try:
        import paths as _paths
        return _paths
    except Exception:
        pass
    import importlib.util
    candidates = [
        Path("paths.py"),
        Path.cwd() / "paths.py",
        Path("_compat/paths.py"),
        Path("./_compat/paths.py"),
        Path("../_compat/paths.py"),
        Path("compat/paths.py"),
        Path("./compat/paths.py"),
        Path("../compat/paths.py"),
        Path("/mnt/data/paths.py"),
    ]
    env_hint = os.getenv("PATHS_PY")
    if env_hint:
        candidates.insert(0, Path(env_hint))
    for p in candidates:
        try:
            if p.exists():
                spec = importlib.util.spec_from_file_location("paths", str(p))
                mod = importlib.util.module_from_spec(spec)
                spec.loader.exec_module(mod)  # type: ignore
                return mod
        except Exception:
            continue
    return None

def _make_paths_shim():
    """Fallback shim if paths.py is unavailable (AC#3: 無ければ自動作成)."""
    class _PathsShim:
        # Decide RUN_ID with env > runreg.bootstrap() > timestamp
        try:
            _rid_env = os.environ.get("RUN_ID")
            if _rid_env:
                RUN_ID = _rid_env
            else:
                try:
                    import run_id_registry as runreg  # optional
                    RUN_ID = runreg.bootstrap() or time.strftime("%Y-%m-%d_%H%M%S")
                except Exception:
                    RUN_ID = time.strftime("%Y-%m-%d_%H%M%S")
        except Exception:
            RUN_ID = time.strftime("%Y-%m-%d_%H%M%S")
        ARTIFACTS_ROOT = Path(os.getenv("ARTIFACTS_ROOT") or "artifacts")
        compat_base_dirs = {
            "root":    ARTIFACTS_ROOT / RUN_ID,
            "handoff": ARTIFACTS_ROOT / RUN_ID / "handoff",
            "logs":    ARTIFACTS_ROOT / RUN_ID / "logs",
            "reports": ARTIFACTS_ROOT / RUN_ID / "reports",
        }
        @classmethod
        def ensure_roots(cls):
            for p in cls.compat_base_dirs.values():
                p.mkdir(parents=True, exist_ok=True)
    print("[04-3] INFO: paths shim in use (paths.py not importable).")
    return _PathsShim

# Resolve paths (prefer project module)
paths = _import_paths()
if paths is None:
    paths = _make_paths_shim()

# Derive RUN_ID (prefer module's value; otherwise env > runreg > timestamp)
try:
    RUN_ID = getattr(paths, "RUN_ID", None)
except Exception:
    RUN_ID = None
if not RUN_ID:
    RUN_ID = os.environ.get("RUN_ID")
if not RUN_ID:
    try:
        import run_id_registry as runreg
        RUN_ID = runreg.bootstrap() or time.strftime("%Y-%m-%d_%H%M%S")
    except Exception:
        RUN_ID = time.strftime("%Y-%m-%d_%H%M%S")

# Handoff dir (prefer module's compat paths)
try:
    handoff_dir = Path(getattr(paths, "compat_base_dirs", {}).get("handoff"))
except Exception:
    handoff_dir = None
if not handoff_dir:
    ARTIFACTS_ROOT = Path(os.getenv("ARTIFACTS_ROOT") or "artifacts")
    handoff_dir = ARTIFACTS_ROOT / RUN_ID / "handoff"
handoff_dir.mkdir(parents=True, exist_ok=True)

PKL_MAIN      = handoff_dir / "04-3_llm_tools_setup_with_tools.pkl"
PKL_WITHTOOLS = handoff_dir / "04-3_llm_tools_setup.pkl"  # legacy / secondary
MANIFEST_JSON = handoff_dir / "04-3_manifest.json"

# ===== Collect variables from previous cells (non-intrusive) =====
def _find_first(names):
    g = globals()
    for n in names:
        if n in g:
            return g[n]
    return None

def _gather_stats():
    st = {}
    if "stats" in globals() and isinstance(globals()["stats"], dict):
        st.update(globals()["stats"])
    bk = _find_first(["brand_keywords","BRAND_KEYWORDS"])
    if bk is not None:
        try: st["brand_keywords"] = list(bk)
        except Exception: st["brand_keywords"] = bk
    cfm = _find_first(["cert_full_info_map","CERT_FULL_INFO_MAP"])
    if cfm is not None:
        st["cert_full_info_map"] = cfm
    hrw = _find_first(["HIGH_RISK_WORDS","high_risk_words"])
    if hrw is not None:
        try: st["HIGH_RISK_WORDS"] = list(hrw) if not isinstance(hrw, dict) else hrw
        except Exception: st["HIGH_RISK_WORDS"] = hrw
    return st

def _prune_tool_obj(obj):
    try:
        name = getattr(obj, "name", None) or getattr(obj, "__name__", None) or type(obj).__name__
        mod  = getattr(obj, "__module__", None)
        desc = getattr(obj, "description", None) or getattr(obj, "__doc__", None)
        try:
            import inspect; sig = str(inspect.signature(getattr(obj, "__call__", obj)))
        except Exception:
            sig = None
        return {"_kind":"tool","name":str(name),"module":str(mod) if mod else None,
                "description": str(desc)[:400] if desc else None,
                "signature": sig, "repr": repr(obj)[:400]}
    except Exception as e:
        return {"_kind":"tool","name":type(obj).__name__,"error":str(e)}

def _prune_tools(tools):
    try:
        if tools is None: return None
        if isinstance(tools, dict):             return {str(k): _prune_tools(v) for k,v in tools.items()}
        if isinstance(tools, (list,tuple,set)): return [_prune_tools(t) for t in tools]
        if callable(tools) or isinstance(tools, object): return _prune_tool_obj(tools)
    except Exception as e:
        return {"_kind":"tools","error":str(e)}
    return tools

def _prune_llm(llm_obj, cfg_llm):
    base = {}
    if isinstance(cfg_llm, dict) and cfg_llm:
        base.update({k: cfg_llm.get(k) for k in ["provider","base_url","model"]})
    if llm_obj is not None:
        base.update({"_type": type(llm_obj).__name__, "repr": repr(llm_obj)[:400]})
    return base or None

def _try_dump(obj, path, use_cloudpickle=True):
    tmp = Path(str(path) + ".tmp")
    try:
        with open(tmp, "wb") as f:
            if use_cloudpickle and _cp is not None:
                _cp.dump(obj, f)
            else:
                _pickle.dump(obj, f, protocol=_pickle.HIGHEST_PROTOCOL)
            f.flush(); os.fsync(f.fileno())
        size = tmp.stat().st_size
        if size <= 0: raise IOError("dump produced 0 bytes")
        tmp.replace(path)
        return True, size, None
    except Exception as e:
        try:
            if tmp.exists(): tmp.unlink()
        except Exception:
            pass
        return False, 0, e

# ===== Build payload (no changes to upstream logic) =====
_cfg = _find_first(["cfg"]) or {}
if not _cfg:
    try:
        with open("/mnt/data/config.json","r",encoding="utf-8") as f:
            _cfg = json.load(f)
    except Exception:
        _cfg = {}

try: _cfg_llm = dict((_cfg or {}).get("llm") or {})
except Exception: _cfg_llm = {}

_tools = _find_first(["tools","TOOLS","agent_tools","toolset","available_tools"])
_llm   = _find_first(["llm","LLM","client","openai_client"])
_stats = _gather_stats()

payload = {"cfg": _cfg, "tools": _tools, "llm": _llm, "stats": _stats}
for key in ["brand_keywords","cert_full_info_map","HIGH_RISK_WORDS"]:
    if key in _stats and key not in payload:
        payload[key] = _stats[key]

# Try save: cloudpickle → prune+cloudpickle → prune+pickle
ok, size, err = _try_dump(payload, PKL_MAIN, use_cloudpickle=True)
if not ok:
    print(f"[04-3] WARN: cloudpickle dump failed: {err}")
    pruned = dict(payload)
    pruned["tools"] = _prune_tools(_tools)
    pruned["llm"]   = _prune_llm(_llm, _cfg_llm)
    ok2, size2, err2 = _try_dump(pruned, PKL_MAIN, use_cloudpickle=True)
    if not ok2:
        print(f"[04-3] WARN: cloudpickle dump (pruned) failed: {err2}")
        ok3, size3, err3 = _try_dump(pruned, PKL_MAIN, use_cloudpickle=False)
        if not ok3:
            raise RuntimeError(f"[04-3] Failed to persist handoff pkl even after pruning: {err3}")
        ok, size, err = ok3, size3, None
    else:
        ok, size, err = ok2, size2, None

# Remove legacy
try:
    if PKL_WITHTOOLS.exists():
        PKL_WITHTOOLS.unlink()
except Exception as e:
    print(f"[04-3] NOTE: Failed to remove legacy: {PKL_WITHTOOLS} ({e})")

# Sanity + Manifest
required_top   = ["cfg","tools","llm","stats"]
required_stats = ["brand_keywords","cert_full_info_map","HIGH_RISK_WORDS"]
present_top    = [k for k in required_top if k in payload and payload[k] is not None]
missing_top    = [k for k in required_top if k not in payload or payload[k] is None]
_stats_keys    = list((_stats or {}).keys()) if isinstance(_stats, dict) else []
present_stats  = [k for k in required_stats if k in (_stats or {})]
missing_stats  = [k for k in required_stats if k not in (_stats or {})]

manifest = {
    "version": "04-3_handoff_v3",
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "run_id": RUN_ID,
    "pkl_path": str(PKL_MAIN),
    "byte_size": int(size),
    "top_level_keys": sorted(list(payload.keys())),
    "stats_keys": sorted(_stats_keys),
    "required_top": required_top,
    "required_stats": required_stats,
    "present_top": present_top,
    "missing_top": missing_top,
    "present_stats": present_stats,
    "missing_stats": missing_stats,
}
with open(MANIFEST_JSON, "w", encoding="utf-8") as f:
    json.dump(manifest, f, ensure_ascii=False, indent=2)

print("[04-3] Handoff (single-file) saved:", PKL_MAIN)
print("[04-3] Size (bytes):", size)
print("[04-3] Present top-level keys:", present_top, "| Missing:", missing_top)
print("[04-3] Present stats keys:", present_stats, "| Missing:", missing_stats)
print("[04-3] Manifest:", MANIFEST_JSON)
print("[04-3] paths module:", getattr(paths, "__name__", type(paths).__name__), "| RUN_ID:", RUN_ID)


[04-3] Handoff (single-file) saved: artifacts/2026-01-03_222059/handoff/04-3_llm_tools_setup_with_tools.pkl
[04-3] Size (bytes): 1023756
[04-3] Present top-level keys: ['cfg', 'stats'] | Missing: ['tools', 'llm']
[04-3] Present stats keys: ['brand_keywords', 'cert_full_info_map'] | Missing: ['HIGH_RISK_WORDS']
[04-3] Manifest: artifacts/2026-01-03_222059/handoff/04-3_manifest.json
[04-3] paths module: _compat.paths | RUN_ID: 2026-01-03_222059


In [16]:
# === 04-3 — 最終セル: handoff 主ファイルの統合保存（ガード付き） ===
# 入力: safe_cfg_dict?, BRAND_KEYWORDS, CERT_FULL_INFO_MAP, TOOLS_DICT, LLM_CONN_INFO?, ASYNC_HINTS?
# 出力: artifacts/{RUN_ID}/handoff/04-3_llm_tools_setup_with_tools.pkl (+ .meta.json)
import os, json, pickle
from pathlib import Path
from datetime import datetime

from _compat.paths import compat_base_dirs

# cfg の確保（Cell1想定、なければ config.json から）
if 'cfg' not in globals():
    try:
        cfg = json.loads(open("config.json","r").read())
        print("[04-3][FIX] cfg を config.json から補完しました。")
    except Exception:
        cfg = {}

# safe_cfg_dict のガード
if 'safe_cfg_dict' not in globals():
    safe_cfg_dict = {
        "run_id": os.environ.get("RUN_ID"),
        "paths": compat_base_dirs,
        "llm": {k: cfg.get("llm", {}).get(k) for k in ["provider","base_url","model"]} if isinstance(cfg, dict) else {},
        "brand_keywords_cfg": (cfg.get("brand_keywords", {}) if isinstance(cfg, dict) else {}),
    }
    print("[04-3][FIX] safe_cfg_dict を生成しました。")

# 最低限の補完（上流が未定義なら作るが、既存があれば尊重）
if 'LLM_CONN_INFO' not in globals():
    LLM_CONN_INFO = {
        "provider": cfg.get("llm", {}).get("provider") if isinstance(cfg, dict) else None,
        "base_url": cfg.get("llm", {}).get("base_url") if isinstance(cfg, dict) else None,
        "model":    cfg.get("llm", {}).get("model")    if isinstance(cfg, dict) else None,
        "api_key_ref": None,
    }
if 'ASYNC_HINTS' not in globals():
    ASYNC_HINTS = {}

REQUIRED_KEYS = ["cfg","brand_keywords","cert_full_info_map","tools","llm","async_client"]

handoff_dir = Path(compat_base_dirs["handoff"])
handoff_dir.mkdir(parents=True, exist_ok=True)

payload = {
    "cfg": safe_cfg_dict,
    "brand_keywords": BRAND_KEYWORDS,          # 上流セルが生成することを前提
    "cert_full_info_map": CERT_FULL_INFO_MAP,  # 同上
    "tools": TOOLS_DICT,                       # 同上
    "llm": LLM_CONN_INFO,
    "async_client": ASYNC_HINTS,
}

out_pkl = handoff_dir / "04-3_llm_tools_setup_with_tools.pkl"
with out_pkl.open("wb") as f:
    pickle.dump(payload, f)

meta = {
    "run_id": os.environ.get("RUN_ID"),
    "created_at": datetime.now().isoformat(timespec="seconds"),
    "file": str(out_pkl), "size": out_pkl.stat().st_size,
    "required_keys": REQUIRED_KEYS,
    "types": {k: type(v).__name__ for k,v in payload.items()},
}
(out_pkl.with_suffix(".meta.json")).write_text(json.dumps(meta, ensure_ascii=False, indent=2))
print("[04-3][FIX] wrote handoff:", out_pkl, "| size =", out_pkl.stat().st_size)

[04-3][FIX] safe_cfg_dict を生成しました。
[04-3][FIX] wrote handoff: artifacts/2026-01-03_222059/handoff/04-3_llm_tools_setup_with_tools.pkl | size = 976815


In [17]:
# === P1-2 TEST (local, no external I/O) ===
if "BRAND_KEYWORDS" not in globals():
    BRAND_KEYWORDS = ["paypal","mercari","ledger"]
res = brand_impersonation_check("paypal-secure-login.info", 0.34)
print("P1-2 test result:", res)
assert isinstance(res, dict) and "confidence" in res and "risk_level" in res and "success" in res
if "paypal" in [k.lower() for k in globals().get("BRAND_KEYWORDS", [])]:
    assert res.get("is_impersonation") is True
print("P1-2 smoketest: OK")


P1-2 test result: {'is_impersonation': True, 'confidence': 0.842, 'risk_level': 'high', 'detected_brands': ['paypal'], 'risk_factors': {'typosquatting_matches': [], 'risk_keywords': ['login', 'secure'], 'ml_probability': 0.34, 'tokens': ['paypal', 'secure', 'login', 'info']}, 'reasons': ['brand keyword present', 'no typo candidate', 'ml_probability contribution=0.102'], 'success': True}
P1-2 smoketest: OK


In [18]:
# === C-01 initialize_llm_client (04-3) ===
# NOTE: This cell was appended by the automation to restore missing functionality
# from 03_ai_agent_analysis.ipynb while keeping minimum diffs.
# - Reads /mnt/data/config.json (llm.provider/base_url/model)
# - Creates a lightweight "client stub" (OpenAI-compatible) without doing network I/O
# - Ensures artifacts/{RUN_ID}/handoff/04-3_llm_tools_setup_with_tools.pkl (+ .meta.json)
#   exist and include required keys used by downstream notebooks (04-4, 04-5).
import os, json, pickle, datetime as _dt

def _load_config(path="/mnt/data/config.json"):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def initialize_llm_client(cfg=None, run_id=None):
    """
    Initialize LLM client with proper error handling (vLLM / OpenAI-compatible).

    Inputs:
        cfg: dict (optional). If None, load from /mnt/data/config.json
        run_id: str (optional). If None, use env RUN_ID or timestamp.

    Returns:
        dict: client_stub = {
            'provider', 'base_url', 'model', 'api_key_ref',
            'client_type', 'async_client', 'version'
        }

    Side effects:
        - Write/Update handoff pickle & meta:
          artifacts/{RUN_ID}/handoff/04-3_llm_tools_setup_with_tools.pkl
          artifacts/{RUN_ID}/handoff/04-3_llm_tools_setup_with_tools.pkl.meta.json
        - Ensure presence of required keys expected by downstream code.
    """
    if cfg is None:
        cfg = _load_config()
    llm_cfg = cfg.get("llm", {})

    provider = llm_cfg.get("provider")
    base_url = llm_cfg.get("base_url")
    model = llm_cfg.get("model")
    api_key_ref = llm_cfg.get("api_key") or None

    # Client stub (no external network; OpenAI-compatible style)
    client_stub = {
        "provider": provider,
        "base_url": base_url,
        "model": model,
        "api_key_ref": "ENV:OPENAI_API_KEY" if (api_key_ref in (None, "", "null")) else api_key_ref,
        "client_type": "openai-compatible",
        "async_client": False,
        "version": "0.1",
    }

    # Prepare handoff paths
    run_id = run_id or os.getenv("RUN_ID") or _dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    art_dir = os.path.join("/mnt/data", "artifacts", run_id, "handoff")
    os.makedirs(art_dir, exist_ok=True)
    pkl_path = os.path.join(art_dir, "04-3_llm_tools_setup_with_tools.pkl")
    meta_path = pkl_path + ".meta.json"

    # Load existing if any
    state = {}
    if os.path.exists(pkl_path):
        try:
            with open(pkl_path, "rb") as f:
                state = pickle.load(f) or {}
        except Exception:
            state = {}

    # Ensure keys presence (keep existing values unless missing)
    def _get(key, default):
        return state[key] if key in state and state[key] not in (None, "") else default

    state.update({
        "provider": provider,
        "base_url": base_url,
        "model": model,
        "llm": client_stub,
        "async_client": _get("async_client", False),
        "brand_keywords": _get("brand_keywords", []),
        "brand_keywords_cfg": _get("brand_keywords_cfg", {}),
        "required_stats": _get("required_stats", []),
        "required_top": _get("required_top", []),
        "present_stats": _get("present_stats", []),
        "present_top": _get("present_top", []),
        "missing_stats": _get("missing_stats", []),
        "missing_top": _get("missing_top", []),
        "stats_keys": _get("stats_keys", []),
        "top_level_keys": _get("top_level_keys", []),
        "paths": _get("paths", {}),
        "run_id": run_id,
        "timestamp": _dt.datetime.now().isoformat(),
        "version": _get("version", "1"),
        "pkl_path": pkl_path,
    })

    # Save, then record byte_size both in meta and state (compat)
    with open(pkl_path, "wb") as f:
        pickle.dump(state, f)
    byte_size = os.path.getsize(pkl_path)
    state["byte_size"] = byte_size
    with open(pkl_path, "wb") as f:
        pickle.dump(state, f)

    meta = {"keys": sorted(list(state.keys())), "byte_size": byte_size, "pkl_path": pkl_path, "run_id": run_id}
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    return client_stub

print("[C-01] initialize_llm_client is ready.")

[C-01] initialize_llm_client is ready.


In [19]:
# === C-03a: Re-export fn_features_df into 04-3 handoff (minimal) ===
import warnings, pickle, os
from pathlib import Path
import pandas as pd

# 標準パス解決
try:
    import importlib, run_id_registry as runreg, _compat.paths as paths
    rid = runreg.bootstrap()
    importlib.reload(paths); 
    paths.ensure_roots()
    RUN_ID = paths.RUN_ID or os.environ.get("RUN_ID")
    HANDOFF_DIR = Path(paths.compat_base_dirs["handoff"])
except Exception as e:
    warnings.warn(f"[04-3] compat paths fallback: {e}")
    RUN_ID = os.environ.get("RUN_ID")
    HANDOFF_DIR = Path(f"artifacts/{RUN_ID}/handoff")

pkl_path = HANDOFF_DIR / "04-3_llm_tools_setup_with_tools.pkl"

# 既存handoffを読み出し
payload = {}
try:
    if pkl_path.exists() and pkl_path.stat().st_size > 0:
        with open(pkl_path, "rb") as f:
            payload = pickle.load(f) or {}
except Exception as e:
    warnings.warn(f"[04-3] load existing handoff failed: {e}; recreate empty")
    payload = {}

def _pick_prob_col(df):
    for c in ["ml_probability","ml_prob","prob","proba","pred_prob","pred_proba","probability","phish_prob","p_phish","score"]:
        if c in getattr(df, "columns", []): 
            return c
    return None

# グローバルに fn_features_df があれば軽量再エクスポート（最大 5000 行）
try:
    if "fn_features_df" in globals():
        _df = globals()["fn_features_df"]
        if isinstance(_df, pd.DataFrame) and "domain" in _df.columns:
            col = _pick_prob_col(_df)
            if col:
                small = _df[["domain", col]].rename(columns={col: "ml_probability"}).copy()
                if len(small) > 5000:
                    small = small.sample(5000, random_state=42)
                payload["fn_features_df"] = small.reset_index(drop=True)
                print(f"[04-3] fn_features_df -> payload (rows={len(small)})")
    else:
        print("[04-3] fn_features_df not found in globals(); skip export")
except Exception as e:
    warnings.warn(f"[04-3] skip re-export fn_features_df: {e}")

# 上書き保存
HANDOFF_DIR.mkdir(parents=True, exist_ok=True)
with open(pkl_path, "wb") as f:
    pickle.dump(payload, f)
print("[04-3] handoff updated:", pkl_path, "| keys:", sorted(payload.keys()))

[04-3] handoff updated: artifacts/2026-01-03_222059/handoff/04-3_llm_tools_setup_with_tools.pkl | keys: ['async_client', 'brand_keywords', 'cert_full_info_map', 'cfg', 'llm', 'tools']


In [20]:
# === C-05-FIX: robust handoff resolver + short_domain_analysis export ===
import os, json, pickle, warnings
from pathlib import Path
import datetime as _dt

def resolve_artifact_dirs(run_id: str = None):
    """
    優先: _compat.paths + run_id_registry → フォールバック: ローカル ./artifacts
    戻り値: (RUN_ID, HANDOFF_DIR(Path), RESULTS_DIR(Path))
    """
    try:
        import importlib, run_id_registry as runreg, _compat.paths as paths
        rid = run_id or runreg.bootstrap()
        importlib.reload(paths); paths.ensure_roots()
        base = paths.compat_base_dirs  # {'handoff': ..., 'results': ..., ...}
        handoff = Path(base["handoff"])
        results = Path(base.get("results", Path(base.get("root", ".")) / "results"))
        return rid, handoff, results
    except Exception as e:
        warnings.warn(f"[resolver] fallback to local ./artifacts (reason: {e})")
        rid = run_id or os.environ.get("RUN_ID") or _dt.datetime.now().strftime("%Y%m%d_%H%M%S")
        root = Path(os.environ.get("ARTIFACTS_ROOT", "artifacts"))
        handoff = root / rid / "handoff"
        results = root / rid / "results"
        handoff.mkdir(parents=True, exist_ok=True)
        results.mkdir(parents=True, exist_ok=True)
        return rid, handoff, results

# short_domain_analysis が未定義なら定義（C-05と同等の最小実装）
try:
    short_domain_analysis
except NameError:
    def short_domain_analysis(domain: str, ml_probability: float) -> dict:
        import math
        def calculate_entropy(text: str) -> float:
            if not text: return 0.0
            probs = [text.count(c)/len(text) for c in set(text)]
            return -sum(p*math.log(p, 2) for p in probs if p > 0)
        d = (domain or "").strip(); dl = d.lower()
        parts = dl.split("."); without_tld = ".".join(parts[:-1]) if len(parts)>1 else dl
        ent = calculate_entropy(without_tld); hyphens = d.count("-")
        digit_ratio = sum(c.isdigit() for c in without_tld) / max(1, len(without_tld))
        res = {"is_phishing": False, "confidence": float(ml_probability), "risk_factors": {}, "reasoning": ""}
        reasons = []
        if len(without_tld) <= 10: res["risk_factors"]["short_name"]=True; res["confidence"]=max(res["confidence"],0.30); reasons.append("短いドメイン名")
        if ent > 4.0: res["risk_factors"]["high_entropy"]=True; res["confidence"]=max(res["confidence"],0.55); reasons.append(f"高エントロピー {ent:.2f}")
        if hyphens > 2: res["risk_factors"]["many_hyphens"]=True; res["confidence"]=max(res["confidence"],0.45); reasons.append(f"ハイフンが多い ({hyphens})")
        if digit_ratio > 0.30: res["risk_factors"]["digit_ratio_high"]=True; res["confidence"]=max(res["confidence"],0.40); reasons.append(f"数字の比率が高い ({digit_ratio:.2f})")
        if dl.startswith("xn--"): res["risk_factors"]["punycode_like"]=True; res["confidence"]=max(res["confidence"],0.60); reasons.append("Punycode っぽい表記")
        if (parts[-1] if len(parts)>1 else "") in {"zip","mov","rest","xyz","top","mom","cam","bar","quest"}:
            res["risk_factors"]["suspicious_tld"]=parts[-1]; res["confidence"]=max(res["confidence"],0.50); reasons.append(f"疑わしいTLD({parts[-1]})")
        res["is_phishing"] = res["confidence"] >= 0.50
        res["reasoning"] = " / ".join(reasons)[:1500] if reasons else "no-issues"
        return res

# handoff に tools_code を追記（既存キーは保持）
SOURCE_short = """{}""".format(short_domain_analysis.__code__.co_consts[0] or "")  # Docstringは空なら空

RUN_ID, HANDOFF_DIR, RESULTS_DIR = resolve_artifact_dirs(os.environ.get("RUN_ID"))
pkl = HANDOFF_DIR / "04-3_llm_tools_setup_with_tools.pkl"
state = {}
if pkl.exists() and pkl.stat().st_size > 0:
    try:
        with open(pkl, "rb") as f: state = pickle.load(f) or {}
    except Exception: state = {}
tools_code = state.get("tools_code", {})
tools_code["short_domain_analysis"] = (
    # ソース文字列が空の場合は安全側の最小版を格納
    '''def short_domain_analysis(domain: str, ml_probability: float) -> dict:
    import math
    def calculate_entropy(text: str) -> float:
        if not text: return 0.0
        probs = [text.count(c)/len(text) for c in set(text)]
        return -sum(p*math.log(p, 2) for p in probs if p > 0)
    d=(domain or "").strip(); dl=d.lower()
    parts=dl.split("."); without_tld=".".join(parts[:-1]) if len(parts)>1 else dl
    ent=calculate_entropy(without_tld); hyphens=d.count("-")
    digit_ratio=sum(c.isdigit() for c in without_tld)/max(1,len(without_tld))
    res={"is_phishing":False,"confidence":float(ml_probability),"risk_factors":{},"reasoning":""}; reasons=[]
    if len(without_tld)<=10: res["risk_factors"]["short_name"]=True; res["confidence"]=max(res["confidence"],0.30); reasons.append("短いドメイン名")
    if ent>4.0: res["risk_factors"]["high_entropy"]=True; res["confidence"]=max(res["confidence"],0.55); reasons.append(f"高エントロピー {ent:.2f}")
    if hyphens>2: res["risk_factors"]["many_hyphens"]=True; res["confidence"]=max(res["confidence"],0.45); reasons.append(f"ハイフンが多い ({hyphens})")
    if digit_ratio>0.30: res["risk_factors"]["digit_ratio_high"]=True; res["confidence"]=max(res["confidence"],0.40); reasons.append(f"数字の比率が高い ({digit_ratio:.2f})")
    if dl.startswith("xn--"): res["risk_factors"]["punycode_like"]=True; res["confidence"]=max(res["confidence"],0.60); reasons.append("Punycode っぽい表記")
    if (parts[-1] if len(parts)>1 else "") in {"zip","mov","rest","xyz","top","mom","cam","bar","quest"}:
        res["risk_factors"]["suspicious_tld"]=parts[-1]; res["confidence"]=max(res["confidence"],0.50); reasons.append(f"疑わしいTLD({parts[-1]})")
    res["is_phishing"]=res["confidence"]>=0.50; res["reasoning"]=" / ".join(reasons)[:1500] if reasons else "no-issues"; return res
    '''
)
state["tools_code"] = tools_code
state.setdefault("present_top", [])
if "short_domain_analysis" not in state["present_top"]:
    state["present_top"].append("short_domain_analysis")
state.setdefault("top_level_keys", [])
for k in ("tools_code","present_top"):
    if k not in state["top_level_keys"]:
        state["top_level_keys"].append(k)
state["timestamp"] = _dt.datetime.now().isoformat()

with open(pkl, "wb") as f: pickle.dump(state, f)
print("[C-05-FIX] HANDOFF_DIR =", HANDOFF_DIR)
print("[C-05-FIX] exported ->", pkl)


[C-05-FIX] HANDOFF_DIR = artifacts/2026-01-03_222059/handoff
[C-05-FIX] exported -> artifacts/2026-01-03_222059/handoff/04-3_llm_tools_setup_with_tools.pkl


In [21]:
# === C-05-REBASE: robust handoff resolver + short_domain_analysis export (04-3) ===
import os, json, pickle, warnings
from pathlib import Path
import datetime as _dt

def resolve_artifact_dirs(run_id: str = None):
    """
    優先: run_id_registry + _compat.paths.ensure_roots() の handoff/results
    フォールバック: カレント配下 ./artifacts/{RUN_ID}/handoff|results
    """
    try:
        import importlib, run_id_registry as runreg, _compat.paths as paths
        rid = run_id or runreg.bootstrap()
        importlib.reload(paths); paths.ensure_roots()
        base = paths.compat_base_dirs           # 例: {'handoff': 'artifacts/<RID>/handoff', 'results': ...}
        handoff = Path(base["handoff"])
        results = Path(base.get("results", Path(base.get("root", ".")) / "results"))
        # 念のため作成（権限やパス不整合時は例外→fallbackへ）
        handoff.mkdir(parents=True, exist_ok=True)
        results.mkdir(parents=True, exist_ok=True)
        return rid, handoff, results
    except Exception as e:
        warnings.warn(f"[resolver] fallback to local ./artifacts (reason: {e})")
        rid = run_id or os.environ.get("RUN_ID") or _dt.datetime.now().strftime("%Y%m%d_%H%M%S")
        root = Path(os.environ.get("ARTIFACTS_ROOT", "artifacts"))
        handoff = root / rid / "handoff"
        results = root / rid / "results"
        handoff.mkdir(parents=True, exist_ok=True)
        results.mkdir(parents=True, exist_ok=True)
        return rid, handoff, results

# `short_domain_analysis` が未定義なら最小実装を定義（03由来・構造化出力保証）
try:
    short_domain_analysis
except NameError:
    def short_domain_analysis(domain: str, ml_probability: float) -> dict:
        import math
        def calculate_entropy(text: str) -> float:
            if not text: return 0.0
            probs = [text.count(c)/len(text) for c in set(text)]
            return -sum(p*math.log(p, 2) for p in probs if p > 0)
        d = (domain or "").strip(); dl = d.lower()
        parts = dl.split("."); without_tld = ".".join(parts[:-1]) if len(parts)>1 else dl
        ent = calculate_entropy(without_tld); hyphens = d.count("-")
        digit_ratio = sum(c.isdigit() for c in without_tld) / max(1, len(without_tld))
        res = {"is_phishing": False, "confidence": float(ml_probability), "risk_factors": {}, "reasoning": ""}
        reasons = []
        if len(without_tld) <= 10: res["risk_factors"]["short_name"]=True; res["confidence"]=max(res["confidence"],0.30); reasons.append("短いドメイン名")
        if ent > 4.0: res["risk_factors"]["high_entropy"]=True; res["confidence"]=max(res["confidence"],0.55); reasons.append(f"高エントロピー {ent:.2f}")
        if hyphens > 2: res["risk_factors"]["many_hyphens"]=True; res["confidence"]=max(res["confidence"],0.45); reasons.append(f"ハイフンが多い ({hyphens})")
        if digit_ratio > 0.30: res["risk_factors"]["digit_ratio_high"]=True; res["confidence"]=max(res["confidence"],0.40); reasons.append(f"数字の比率が高い ({digit_ratio:.2f})")
        if dl.startswith("xn--"): res["risk_factors"]["punycode_like"]=True; res["confidence"]=max(res["confidence"],0.60); reasons.append("Punycode っぽい表記")
        if (parts[-1] if len(parts)>1 else "") in {"zip","mov","rest","xyz","top","mom","cam","bar","quest"}:
            res["risk_factors"]["suspicious_tld"]=parts[-1]; res["confidence"]=max(res["confidence"],0.50); reasons.append(f"疑わしいTLD({parts[-1]})")
        res["is_phishing"] = res["confidence"] >= 0.50
        res["reasoning"] = " / ".join(reasons)[:1500] if reasons else "no-issues"
        return res

# handoffに tools_code として同梱（既存キーは保持・非破壊）
# ここでは関数本体を安全な最小版として文字列化（環境差のない exec 用）
SOURCE_short = '''
def short_domain_analysis(domain: str, ml_probability: float) -> dict:
    import math
    def calculate_entropy(text: str) -> float:
        if not text: return 0.0
        probs = [text.count(c)/len(text) for c in set(text)]
        return -sum(p*math.log(p, 2) for p in probs if p > 0)
    d=(domain or "").strip(); dl=d.lower()
    parts=dl.split("."); without_tld=".".join(parts[:-1]) if len(parts)>1 else dl
    ent=calculate_entropy(without_tld); hyphens=d.count("-")
    digit_ratio=sum(c.isdigit() for c in without_tld)/max(1,len(without_tld))
    res={"is_phishing":False,"confidence":float(ml_probability),"risk_factors":{},"reasoning":""}; reasons=[]
    if len(without_tld)<=10: res["risk_factors"]["short_name"]=True; res["confidence"]=max(res["confidence"],0.30); reasons.append("短いドメイン名")
    if ent>4.0: res["risk_factors"]["high_entropy"]=True; res["confidence"]=max(res["confidence"],0.55); reasons.append(f"高エントロピー {ent:.2f}")
    if hyphens>2: res["risk_factors"]["many_hyphens"]=True; res["confidence"]=max(res["confidence"],0.45); reasons.append(f"ハイフンが多い ({hyphens})")
    if digit_ratio>0.30: res["risk_factors"]["digit_ratio_high"]=True; res["confidence"]=max(res["confidence"],0.40); reasons.append(f"数字の比率が高い ({digit_ratio:.2f})")
    if dl.startswith("xn--"): res["risk_factors"]["punycode_like"]=True; res["confidence"]=max(res["confidence"],0.60); reasons.append("Punycode っぽい表記")
    if (parts[-1] if len(parts)>1 else "") in {"zip","mov","rest","xyz","top","mom","cam","bar","quest"}:
        res["risk_factors"]["suspicious_tld"]=parts[-1]; res["confidence"]=max(res["confidence"],0.50); reasons.append(f"疑わしいTLD({parts[-1]})")
    res["is_phishing"]=res["confidence"]>=0.50; res["reasoning"]=" / ".join(reasons)[:1500] if reasons else "no-issues"
    return res
'''.strip()

RUN_ID, HANDOFF_DIR, RESULTS_DIR = resolve_artifact_dirs(os.environ.get("RUN_ID"))
pkl = HANDOFF_DIR / "04-3_llm_tools_setup_with_tools.pkl"

state = {}
if pkl.exists() and pkl.stat().st_size > 0:
    try:
        with open(pkl, "rb") as f:
            state = pickle.load(f) or {}
    except Exception as e:
        warnings.warn(f"[04-3] read handoff failed, recreate empty: {e}")
        state = {}

tools_code = state.get("tools_code", {})
tools_code["short_domain_analysis"] = SOURCE_short
state["tools_code"] = tools_code

state.setdefault("present_top", [])
if "short_domain_analysis" not in state["present_top"]:
    state["present_top"].append("short_domain_analysis")

state.setdefault("top_level_keys", [])
for k in ("tools_code", "present_top"):
    if k not in state["top_level_keys"]:
        state["top_level_keys"].append(k)

state["timestamp"] = _dt.datetime.now().isoformat()

with open(pkl, "wb") as f:
    pickle.dump(state, f)

print("[C-05-REBASE] RUN_ID      =", RUN_ID)
print("[C-05-REBASE] HANDOFF_DIR =", HANDOFF_DIR)
print("[C-05-REBASE] exported    ->", pkl)
print("[C-05-REBASE] keys        ->", sorted(state.keys()))
print("[C-05-REBASE] tools_code  ->", sorted(state.get("tools_code",{}).keys()))


[C-05-REBASE] RUN_ID      = 2026-01-03_222059
[C-05-REBASE] HANDOFF_DIR = artifacts/2026-01-03_222059/handoff
[C-05-REBASE] exported    -> artifacts/2026-01-03_222059/handoff/04-3_llm_tools_setup_with_tools.pkl
[C-05-REBASE] keys        -> ['async_client', 'brand_keywords', 'cert_full_info_map', 'cfg', 'llm', 'present_top', 'timestamp', 'tools', 'tools_code', 'top_level_keys']
[C-05-REBASE] tools_code  -> ['short_domain_analysis']
